# Imports

In [190]:
import gurobipy as gp
from gurobipy import GRB, quicksum, max_
import random
from time import time
random.seed(10)

# Utils

In [191]:
def gsk(dict):
    return sorted(list(dict.keys()))

# Inputs

In [192]:
slots = list(range(0, 20))
campuses = list(range(0, 2))
courses = {i: {"campus": random.choice(campuses), "is_lab": random.uniform(0, 1)>0.85} for i in range(0, 8)}
students = {i: set(random.sample(gsk(courses), random.randint(3, 6))) for i in range(0, 30)}
venues = {i: {"campus": random.choice(campuses), "capacity": random.randint(20, 40)} for i in range(0, 10)}
print("Data Created")

Data Created


# Model

In [193]:
model = gp.Model("Midsem_Scheduling")
print("Model Created")
X = {}
for slot in slots:
    for course in gsk(courses):
        for venue in gsk(venues):
            for student in gsk(students):
                X[slot, course, venue, student] = model.addVar(vtype=GRB.BINARY, name=f'X_{slot}_{course}_{venue}_{student}')
print(f"Decision Variables Created: {len(X)}")

Model Created
Decision Variables Created: 48000


# Constraints

### 1. Each student must have exactly 1 exam for each course enrolled in, and 0 if not enrolled

In [194]:
for student in gsk(students):
    for course in gsk(courses):
        model.addConstr(quicksum(X[slot, course, venue, student] 
                                    for slot in slots
                                    for venue in gsk(venues)) == int(course in students[student]),
                        name=f'1_{student}_{course}')

### 2. Theory courses are scheduled in a single slot

In [195]:
slot_course_scheduled = {}
for course in gsk(courses):
    for slot in slots:
        slot_course_scheduled[course, slot] = model.addVar(vtype=GRB.BINARY, name=f'slot_course_scheduled_{slot}_{course}')
        
for course in gsk(courses):
    for slot in slots:
        model.addConstr(slot_course_scheduled[course, slot] == max_(X[slot, course, venue, student]
                                                                    for venue in gsk(venues)
                                                                    for student in gsk(students)),
                        name=f'2_{course}_{slot}')

for course in courses:
    if courses[course]["is_lab"]:
        continue
    total = gp.quicksum(X[slot, course, venue, student]
                        for venue in venues
                        for student in students
                        for slot in slots)
    for slot in slots:
        sub_sum = gp.quicksum(X[slot, course, venue, student]
                              for venue in venues
                              for student in students)
        model.addConstr(slot_course_scheduled[course, slot]*total == slot_course_scheduled[course, slot]*sub_sum, name=f'3_constraint_{course}_{slot}')

### 3. Total strength should not exceed venue capacity

In [196]:
for slot in slots:
    for venue in gsk(venues):
        model.addConstr(quicksum(X[slot, course, venue, student]
                                 for course in gsk(courses)
                                 for student in gsk(students)) <= venues[venue]["capacity"],
                        name=f'4_{slot}_{venue}')

### 4. Each course must be sheduled at the designated campus

In [197]:
for course in gsk(courses):
    course_campus = courses[course]["campus"]
    for venue in gsk(venues):
        venue_campus = venues[venue]["campus"]
        if course_campus != venue_campus:
            model.addConstr(quicksum(X[slot, course, venue, student]
                                        for student in gsk(students)
                                        for slot in slots) == 0,
                            name=f'5_{course}_{venue}')

### 5. Each student has atmost one exam per day

In [198]:
for student in students:
    for slot in slots:
        if slot%2==0:
            model.addConstr(quicksum(X[s, course, venue, student] 
                                    for course in gsk(courses)
                                    for venue in gsk(venues)
                                    for s in range(slot, slot+2)) <= 1,
                            name=f'6_{student}_{slot}')

### 6. In the one slot and one venue there cant be multiple courses scheduled

In [199]:
slot_venue_course_scheduled = {}
for slot in slots:
    for venue in gsk(venues):
        for course in gsk(courses):
            slot_venue_course_scheduled[slot, venue, course] = model.addVar(vtype=GRB.BINARY, name=f'slot_venue_course_scheduled_{slot}_{venue}_{course}')
        
for slot in slots:
    for venue in gsk(venues):
        for course in gsk(courses):
            model.addConstr(slot_venue_course_scheduled[slot, venue, course] == max_(X[slot, course, venue, student]
                                                                        for student in gsk(students)),
                            name=f'7_{slot}_{venue}_{course}')

for slot in slots:
    for venue in gsk(venues):
        model.addConstr(quicksum(slot_venue_course_scheduled[slot, venue, course]
                                 for course in gsk(courses)) <= 1,
                        name=f'8_{slot}_{venue}')

# Objective
### Minimize number of slots used

In [200]:
slot_scheduled = {}
for slot in slots:
    slot_scheduled[slot] = model.addVar(vtype=GRB.BINARY, name=f'slot_scheduled_{slot}')

for slot in slots:
    model.addConstr(slot_scheduled[slot] == max_(slot_course_scheduled[course, slot] for course in gsk(courses)),
                    name=f'objective_{slot}')

model.setObjective(
    quicksum(slot_scheduled[slot] for slot in slots),
    GRB.MINIMIZE )

# Optimize

In [201]:
start_time = time()
max_time = 5*60 # 5 minutes

# Terminate the model after 5 minutes
def model_callback(model, where):
    if max_time < time() - start_time:
        model.terminate()

model.optimize(callback=model_callback)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 982 rows, 49780 columns and 170800 nonzeros
Model fingerprint: 0x28724614
Model has 140 quadratic constraints
Model has 1780 general constraints
Variable types: 0 continuous, 49780 integer (49780 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 11033 rows and 35080 columns (presolve time = 5s) ...
Presolve added 246327 rows and 0 columns
Presolve removed 0 rows and 35080 columns
Presolve time: 6.70s
Presolved: 247309 rows, 14700 columns, 551780 nonzeros
Variable types: 0 continuous, 14700 integer (14700 binary)
Found heuristic solution: objective 17.0000000

Root simplex log..

# Parse the Model

In [203]:
def parse_solution(model):
    scheduled_exams = []

    # Iterate through the decision variables (X) and collect scheduled exams
    for slot in slots:
        for course in courses:
            for venue in venues:
                for student in students:
                    if X[slot, course, venue, student].x > 0:
                        scheduled_exams.append((slot, course, venue, student))

    seen_slot = set()
    seen_course = set()
    with open("schedule.txt", "w") as file:
        for slot, course, venue, student in scheduled_exams:
            if not slot in seen_slot:
                seen_slot.add(slot)
                file.write("-------------------------------------------------\n")
                file.write("\n")
                file.write("\n")
                file.write(f"--------------------- SLOT-{slot} --------------------\n")
                file.write("\n")
            if not (slot, course) in seen_course:
                seen_course.add((slot, course))
                file.write("-------------------------------------------------\n")
                lab = "THRY" if not courses[course]["is_lab"] else "LABS"
                course_name = f"Course Name ({lab}): {course}"
                file.write(f"| {course_name:<45} |\n")
                file.write("-------------------------------------------------\n")
            student_venue = f"Student: {student:<13} | Venue: {venue:<13}"
            file.write(f"| {student_venue} |\n")
        file.write("-------------------------------------------------\n")


parse_solution(model)
